## GRADCAM 

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torch.nn.functional as F
import numpy as np
from torchvision.transforms import Compose,ToTensor,Normalize
from torchvision.models import VGG,resnet34
from torch.utils.data import DataLoader

In [9]:
data = dsets.CIFAR10(root='./',transform= Compose([ToTensor()]),download=True,train=True)

Extracting ./cifar-10-python.tar.gz to ./


In [15]:
u,s=data.data.mean((0,1,2))/255, data.data.std((0,1,2))/255

In [19]:
u,s

(array([0.49139968, 0.48215841, 0.44653091]),
 array([0.24703223, 0.24348513, 0.26158784]))

In [20]:
data = dsets.CIFAR10(root='./',transform= Compose([ToTensor(),Normalize(mean=u,std=s)]),download=True,train=True)

Files already downloaded and verified


In [22]:
VGG?VGG